# Game to Movie Video Conversion

## Main Imports

In [38]:
import natsort
import cv2
import random
from os import listdir

## Preprocess the video data
### Function to read frames from files

In [26]:
def read_frames(video_path, save_path, problem_frames, start_frame, end_frame, frame_skip, counter):
    
    # initialise frame capturing
    vidcap = cv2.VideoCapture(video_path)
    vidcap.set(1, start_frame)
    success, image = vidcap.read()
    frame_number = start_frame
    
    while success and frame_number < end_frame:

        # save frames as images, capturing every frame_skip frames
        cv2.imwrite(save_path + '%d.jpg' % counter, image)
        frame_number += frame_skip
        counter += 1
        
        if frame_number in problem_frames:
            original_frame_number = frame_number
            random_frame = random.randint(start_frame, end_frame)
            vidcap.set(1, random_frame)
            frame_number = original_frame_number
        else:
            vidcap.set(1, frame_number)
        success, image = vidcap.read()

### Initialise file paths

In [23]:
game_load_path = 'data/game/MafiaVideogame.mp4'
game_save_path = 'data/game/frames/'
game_problem_frames = [11985, 22485, 23610, 34485, 49485, 49860, 65985, 80235, 89235, 103485, 103860, 116235, 138360, 147735,
                       154860, 166860, 180360, 182235, 191985, 202485, 221610, 229110]

movies_folder = 'data/movie/'
movie_save_path = 'data/movie/frames/'

# data holding start frame, end frame, frame skip
movie_data = {
    "TheGodfather.mp4": (204, 12180, 60, [1104, 1164, 5604]),
    "TheIrishman.mp4": (175, 23175, 115, [12710]),
    "TheSopranos.mp4": (80, 51280, 256, [])
}

### Read frames from game file

In [24]:
read_frames(game_load_path, game_save_path, game_problem_frames, 7110, 232110, 375, counter=1)

### Read frames from movie files

In [27]:
counter = 1
for movie_name, frame_data in movie_data.items():
    
    movie_load_path = movies_folder + movie_name
    start_frame, end_frame, frame_skip, movie_problem_frames = frame_data[0], frame_data[1], frame_data[2], frame_data[3]
    read_frames(movie_load_path, movie_save_path, movie_problem_frames, start_frame, end_frame, frame_skip, counter)
    counter += 200